# imports

In [1]:
import pandas as pd
import os
import json
import overpy
import pandas as pd
import geopandas as gpd
import h3
from shapely.geometry import Point, mapping, MultiPoint
import folium

#turning off shapely warnings
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning)

get_working_dir = os.getcwd()
get_working_dir

'/home/laia/code/Laiagdla/notebooks_git/Livables_notebooks'

# read csv 

In [2]:
query_df = pd.read_csv(os.path.join(get_working_dir, 'data', 'queries.csv'))

In [3]:
# pd.set_option('display.max_rows', None)
query_df

,category,sub_cat,collection,feature,active,node,way,rel
0,mobility,public_transport_bus,amenity,bus_station,1.0,1.0,NaN,NaN
1,mobility,public_transport_bus,bus_bay,NaN,1.0,1.0,NaN,NaN
2,mobility,public_transport_bus,highway,bus_stop,1.0,1.0,NaN,NaN
3,mobility,railway,railway,light_rail,1.0,1.0,NaN,NaN
4,mobility,railway,bridge,viaduct,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
129,comfort,rivers,waterway,river,NaN,NaN,1.0,NaN
130,NaN,industrial,building,industrial,NaN,NaN,NaN,NaN
131,NaN,warehouse,building,warehouse,NaN,NaN,NaN,NaN
132,NaN,NaN,historic,sub_cat,NaN,NaN,NaN,NaN


# query builds

In [5]:
# open testing json file
city_name = 'Kreuzberg'

file_name= os.path.join(get_working_dir, 'data', f'{city_name}_nominatim.json')
f = open(file_name)
city_data = json.load(f)
osm_id = city_data['osm_id'] + 3600000000
osm_id

3600055765

In [6]:
columns = query_df.columns

In [7]:
query_df_nodes = query_df.loc[query_df['node'] == 1,columns]

In [8]:
query_df_nodes

,category,sub_cat,collection,feature,active,node,way,rel
0,mobility,public_transport_bus,amenity,bus_station,1.0,1.0,NaN,NaN
1,mobility,public_transport_bus,bus_bay,NaN,1.0,1.0,NaN,NaN
2,mobility,public_transport_bus,highway,bus_stop,1.0,1.0,NaN,NaN
3,mobility,railway,railway,light_rail,1.0,1.0,NaN,NaN
4,mobility,railway,bridge,viaduct,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
101,comfort,leisure_mass,leisure,bandstand,NaN,1.0,NaN,NaN
102,comfort,leisure_mass,leisure,swimming_pool,NaN,1.0,NaN,NaN
103,comfort,leisure_mass,leisure,stadium,NaN,1.0,NaN,NaN
104,comfort,leisure_mass,leisure,sports_centre,NaN,1.0,NaN,NaN


In [23]:
# [out:json][timeout:25];
# area(id:3600055765)->.searchArea;(
# node["leisure"="park"](area.searchArea);
# way["leisure"="park"](area.searchArea);
# relation["leisure"="park"](area.searchArea);
# );out geom;>;out skel qt;

# [out:json][timeout:25];area(id:3600055765)->.searchArea;(
    # node["leisure"="park"](area.searchArea);
    # way["leisure"="park"](area.searchArea);
    # relation["leisure"="park"](area.searchArea);
    # );out geom;>;out skel qt;
    
collection = 'amenity'
feature = 'kindergarten'

query_init = '[out:json][timeout:25];'
query_area = f'area(id:{osm_id})->.searchArea;('
query_nodes = f'node["{collection}"="{feature}"](area.searchArea);'
query_ways = f'way["{collection}"="{feature}"](area.searchArea);'
query_relations = f'relation["{collection}"="{feature}"](area.searchArea);'
query_output_center = ');out center;'
query_output_all = ');out body;>;out skel qt;'

# Get city boundaries
query = query_init + query_area + query_nodes + query_ways + query_relations + query_output_center
api = overpy.Overpass()
result = api.query(query)

In [24]:
print(query)

[out:json][timeout:25];area(id:3600055765)->.searchArea;(node["amenity"="kindergarten"](area.searchArea);way["amenity"="kindergarten"](area.searchArea);relation["amenity"="kindergarten"](area.searchArea););out center;


In [25]:
len(result.nodes)

118

In [12]:
def query_constructor(collection, feature, osm_id):
    query_init = '[out:json][timeout:25];' # set output format and timeout
    query_area = f'area(id:{osm_id})->.searchArea;(' # set area to search in
    query_nodes = f'node["{collection}"="{feature}"](area.searchArea);' # search for nodes
    query_ways = f'way["{collection}"="{feature}"](area.searchArea);' # search for ways
    query_relations = f'relation["{collection}"="{feature}"](area.searchArea);'# search for relations
    query_output = ');out body;>;out skel qt;' # set output format
    
    query = query_init + query_area + query_nodes + query_ways + query_relations + query_output
    return query

In [13]:
for i, row in query_df_nodes.iterrows():
    query = query_constructor(row['collection'], row['feature'], osm_id)
    print(query)
    break

[out:json][timeout:25];area(id:3600055765)->.searchArea;(node["amenity"="bus_station"](area.searchArea);way["amenity"="bus_station"](area.searchArea);relation["amenity"="bus_station"](area.searchArea););out body;>;out skel qt;


In [14]:
city_name = "Kreuzberg"

city_hex_gpd = gpd.read_file(f'{get_working_dir}/data/{city_name}_hexagons.gpkg')
city_hex_gpd

,hex_id,_merge,geometry
0,8a1f18b25867fff,1.0,"POLYGON ((13.41245 52.50037, 13.41213 52.49976..."
1,8a1f18b25477fff,1.0,"POLYGON ((13.40441 52.48684, 13.40409 52.48624..."
2,8a1f18b2499ffff,1.0,"POLYGON ((13.36785 52.49553, 13.36754 52.49492..."
3,8a1f18b24967fff,1.0,"POLYGON ((13.37436 52.50051, 13.37404 52.49990..."
4,8a1f1d4d679ffff,1.0,"POLYGON ((13.42578 52.50153, 13.42546 52.50092..."
...,...,...,...
1327,8a1f18b2096ffff,0.0,"POLYGON ((13.39519 52.48417, 13.39487 52.48356..."
1328,8a1f1d4d602ffff,0.0,"POLYGON ((13.42869 52.51086, 13.42838 52.51026..."
1329,8a1f1d4d2147fff,0.0,"POLYGON ((13.44864 52.49657, 13.44832 52.49597..."
1330,8a1f1d48bc47fff,0.0,"POLYGON ((13.38415 52.50991, 13.38384 52.50930..."


In [15]:
query_df_nodes.columns.values.tolist()[3]

'feature'

In [19]:
res = 10
from tqdm.auto import tqdm
import time


df_list = []
# main_gdf = city_hex_gpd.copy()

for i in tqdm(range(len(query_df_nodes.head(2)))):
    for i, row in query_df_nodes.head(2).iterrows():
        
        query = query_constructor(row['collection'], row['feature'], osm_id)
        api = overpy.Overpass()
        result = api.query(query)   
        
        nodes = []
        hex_id = []
        df_name = f"{row['collection']}_{row['feature']}"
        # nodes_dict = {'geometry':[],'hex_id':[]}
        
        nodes = [Point(node.lat, node.lon) for node in result.nodes]
        hex_id = [h3.geo_to_h3( node.lat, node.lon, res  ) for node in result.nodes]
        
        nodes_df = pd.DataFrame({ df_name : nodes, 'hex_id': hex_id})
        # # nodes_count_df = nodes_df.groupby( nodes_df['hex_id'], as_index=False).count()
        globals()[df_name] = nodes_df.groupby( nodes_df['hex_id']).count()
        df_list.append(globals()[df_name])
        # main_gdf = pd.concat([main_gdf,nodes_count_df], join='outer', axis=1)
        # time.sleep(20)
        # main_gdf = pd.merge(main_gdf, nodes_count_df, how='left', on='hex_id')
        

  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
result.nodes

[]

In [21]:
df_list

[Empty DataFrame
 Columns: [amenity_bus_station]
 Index: [],
 Empty DataFrame
 Columns: [bus_bay_nan]
 Index: [],
 Empty DataFrame
 Columns: [amenity_bus_station]
 Index: [],
 Empty DataFrame
 Columns: [bus_bay_nan]
 Index: []]

In [24]:
main_gdf = city_hex_gpd.copy()

for df in df_list:
    main_gdf.merge(df, on='hex_id', how='outer')

In [22]:
main_gdf

,hex_id,_merge,geometry
0,8a1f18b25867fff,1.0,"POLYGON ((13.41245 52.50037, 13.41213 52.49976..."
1,8a1f18b25477fff,1.0,"POLYGON ((13.40441 52.48684, 13.40409 52.48624..."
2,8a1f18b2499ffff,1.0,"POLYGON ((13.36785 52.49553, 13.36754 52.49492..."
3,8a1f18b24967fff,1.0,"POLYGON ((13.37436 52.50051, 13.37404 52.49990..."
4,8a1f1d4d679ffff,1.0,"POLYGON ((13.42578 52.50153, 13.42546 52.50092..."
...,...,...,...
1327,8a1f18b2096ffff,0.0,"POLYGON ((13.39519 52.48417, 13.39487 52.48356..."
1328,8a1f1d4d602ffff,0.0,"POLYGON ((13.42869 52.51086, 13.42838 52.51026..."
1329,8a1f1d4d2147fff,0.0,"POLYGON ((13.44864 52.49657, 13.44832 52.49597..."
1330,8a1f1d48bc47fff,0.0,"POLYGON ((13.38415 52.50991, 13.38384 52.50930..."


In [ ]:
nodes_df
nodes_count_df = nodes_df.groupby( nodes_df['hex_id']).count()
nodes_count_df

,bridge_viaduct
hex_id,
8a1f18b24827fff,18
8a1f18b24837fff,3
8a1f18b24927fff,2
8a1f18b2492ffff,1
8a1f18b24947fff,16
8a1f18b2494ffff,5
8a1f18b24957fff,7
8a1f18b24967fff,17
8a1f18b2496ffff,7


In [ ]:
main_gdf

,_merge,geometry,amenity_bus_station,bus_bay_nan,highway_bus_stop,railway_light_rail,bridge_viaduct,amenity_bus_station,bus_bay_nan,highway_bus_stop,...,amenity_bus_station,bus_bay_nan,highway_bus_stop,railway_light_rail,bridge_viaduct,amenity_bus_station,bus_bay_nan,highway_bus_stop,railway_light_rail,bridge_viaduct
hex_id,,,,,,,,,,,,,,,,,,,,,
8a1f18b25867fff,1.0,"POLYGON ((13.41245 52.50037, 13.41213 52.49976...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a1f18b25477fff,1.0,"POLYGON ((13.40441 52.48684, 13.40409 52.48624...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a1f18b2499ffff,1.0,"POLYGON ((13.36785 52.49553, 13.36754 52.49492...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a1f18b24967fff,1.0,"POLYGON ((13.37436 52.50051, 13.37404 52.49990...",NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,17.0
8a1f1d4d679ffff,1.0,"POLYGON ((13.42578 52.50153, 13.42546 52.50092...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1f18b2096ffff,0.0,"POLYGON ((13.39519 52.48417, 13.39487 52.48356...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a1f1d4d602ffff,0.0,"POLYGON ((13.42869 52.51086, 13.42838 52.51026...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a1f1d4d2147fff,0.0,"POLYGON ((13.44864 52.49657, 13.44832 52.49597...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cit = city_hex_gpd.copy().set_index('hex_id')
concat = pd.concat([cit,nodes_count_df], join='outer', axis=1)
concat
concat_2 = pd.concat([concat,nodes_count_df], join='outer', axis=1)
concat_2

,_merge,geometry,bridge_viaduct,bridge_viaduct
hex_id,,,,
8a1f18b25867fff,1.0,"POLYGON ((13.41245 52.50037, 13.41213 52.49976...",NaN,NaN
8a1f18b25477fff,1.0,"POLYGON ((13.40441 52.48684, 13.40409 52.48624...",NaN,NaN
8a1f18b2499ffff,1.0,"POLYGON ((13.36785 52.49553, 13.36754 52.49492...",NaN,NaN
8a1f18b24967fff,1.0,"POLYGON ((13.37436 52.50051, 13.37404 52.49990...",17.0,17.0
8a1f1d4d679ffff,1.0,"POLYGON ((13.42578 52.50153, 13.42546 52.50092...",NaN,NaN
...,...,...,...,...
8a1f18b2096ffff,0.0,"POLYGON ((13.39519 52.48417, 13.39487 52.48356...",NaN,NaN
8a1f1d4d602ffff,0.0,"POLYGON ((13.42869 52.51086, 13.42838 52.51026...",NaN,NaN
8a1f1d4d2147fff,0.0,"POLYGON ((13.44864 52.49657, 13.44832 52.49597...",NaN,NaN


In [ ]:
main_df['sum'] = 

In [ ]:
nodes = []
hex_id = []
nodes = [Point(node.lat, node.lon) for node in result.nodes]
hex_id = [h3.geo_to_h3(node.lat, node.lon,11) for node in result.nodes]

In [ ]:
len(nodes)

118

In [ ]:
len(nodes) == len(hex_id)

True

In [ ]:
nodes_ps = pd.Series(nodes)
hex_id_ps = pd.Series(hex_id)

nodes_df = pd.DataFrame({'geometry': nodes, 'hex_id': hex_id})
# nodes_gdf = gpd.GeoDataFrame({'nodes': nodes, 'hex_id': hex_id}, geometry=nodes)

/home/laia/.pyenv/versions/3.8.12/envs/livablestreetsenv/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [ ]:
nodes_count_df = nodes_df.groupby(nodes_df['hex_id']).count()

In [ ]:
nodes_count_df

,geometry
hex_id,
8b1f18b24003fff,1
8b1f18b24014fff,2
8b1f18b24075fff,1
8b1f18b240a1fff,1
8b1f18b240abfff,1
...,...
8b1f1d4d66b2fff,1
8b1f1d4d66c4fff,2
8b1f1d4d66e5fff,1


In [ ]:
nodes_df[f'{collection}_{feature}'] = 1

In [ ]:
nodes_df.drop(columns='geometry', inplace=True)

In [ ]:
nodes_df

,hex_id,amenity_kindergarten
0,8b1f1d4d66e5fff,1
1,8b1f18b25c74fff,1
2,8b1f18b25ab2fff,1
3,8b1f18b2552efff,1
4,8b1f18b24242fff,1
...,...,...
113,8b1f18b25b65fff,1
114,8b1f1d48b28dfff,1
115,8b1f1d48b46afff,1
116,8b1f18b250c5fff,1


In [ ]:
main_gdf = pd.merge(city_hex_gpd, nodes_df,  how='outer', on='hex_id').fillna(0, inplace = True) 


In [ ]:
main_gdf

,hex_id,_merge,geometry,amenity_kindergarten
0,8b1f18b25071fff,1.0,"POLYGON ((13.41525 52.49208, 13.41527 52.49185...",NaN
1,8b1f1d48b696fff,1.0,"POLYGON ((13.39095 52.49694, 13.39097 52.49671...",NaN
2,8b1f1d48b6dafff,1.0,"POLYGON ((13.39822 52.49668, 13.39824 52.49645...",NaN
3,8b1f1d48b389fff,1.0,"POLYGON ((13.40263 52.50540, 13.40265 52.50517...",NaN
4,8b1f1d48b4d1fff,1.0,"POLYGON ((13.38463 52.49902, 13.38464 52.49879...",NaN
...,...,...,...,...
9340,8b1f18b252d9fff,0.0,"POLYGON ((13.42800 52.48652, 13.42802 52.48628...",NaN
9341,8b1f1d48ba84fff,0.0,"POLYGON ((13.39932 52.51084, 13.39933 52.51060...",NaN
9342,8b1f1d48baa0fff,0.0,"POLYGON ((13.39926 52.51154, 13.39928 52.51130...",NaN
9343,8b1f1d48b8ddfff,0.0,"POLYGON ((13.39271 52.51184, 13.39273 52.51161...",NaN


In [ ]:
main_gdf.describe()

,_merge,amenity_kindergarten
count,9345.000000,118.0
mean,0.581059,1.0
std,0.493412,0.0
min,0.000000,1.0
25%,0.000000,1.0
50%,1.000000,1.0
75%,1.000000,1.0
max,1.000000,1.0


In [ ]:
nodes_geojson = MultiPoint(nodes)


In [ ]:

map_test = folium.Map(width=750,
                      height=500,
                      location=[
                          city_hex_gpd.geometry[1].centroid.y,#lat
                          city_hex_gpd.geometry[1].centroid.x,#lon
                        # result.nodes[0].lat,
                        # result.nodes[0].lon
                      ],
                      tiles="openstreetmap",
                      zoom_start=13)

folium.GeoJson(nodes_geojson.__geo_interface__).add_to(map_test)
folium.GeoJson(city_hex_gpd.geometry[1]).add_to(map_test)

https://pydeck.gl/layer.html

In [ ]:
"""
H3HexagonLayer
==============

Plot of values for a particular hex ID in the H3 geohashing scheme.

This example is adapted from the deck.gl documentation.
"""

import pydeck as pdk
import pandas as pd

# H3_HEX_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf.h3cells.json"

# df = pd.read_json(H3_HEX_DATA)
df = main_gdf
# df = nodes_count_df

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="hex_id",
    opacity=0.3,
    get_fill_color="[255 - amenity_kindergarten*255, 150, amenity_kindergarten*255]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=0.1,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=city_hex_gpd.geometry[1].centroid.y, longitude=city_hex_gpd.geometry[1].centroid.x, zoom=14, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Count: {amenity_kindergarten}"})
r.to_html("h3_hexagon_layer.html")